In [16]:
!pip install pyreadr
import pyreadr
import pandas as pd
import numpy as np

#visualisations
import matplotlib.pyplot as plt
import seaborn as sns

#models
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score

from scipy.stats import loguniform, reciprocal

import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [17]:
#Connnect to google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
## Import librabries
import sys
import os

py_file_location = "/content/gdrive/My Drive/ML-exam/dataprocessing"
sys.path.append(os.path.abspath(py_file_location))

import preprocessing as pp

In [19]:
#loading hiv and tubercolosis data
data_hiv = pyreadr.read_r('/content/gdrive/MyDrive/ML-exam/dataprocessing/dataHIV.Rdata')['dfHIV']
data_tb = pyreadr.read_r('/content/gdrive/MyDrive/ML-exam/dataprocessing/dataTB.RData')['dfTB']
df = data_tb.copy().append(data_hiv.copy())

print(data_hiv.shape)
print(data_tb.shape)
print(df.shape)

# imputing nan value for ClaimSec
df.loc[df[df.aucID == "0306500000117000001"].index,"claimSec"] = 1611.80
df.loc[df[df.aucID == "0356200006718000097"].index,"claimSec"] = 424.1

# imputing nan value for bvSameBO
df.loc[df[df.aucID == "0888500000219000457"].index,"bvSameBO"] = 1
df = pp.encode_variables(df)
df = pp.filter_data(df)
df = df.drop(columns=['aucDateOR'])
df = pp.scale_variables(df)
print('Before handling missing values', df.shape)

df = df.dropna().copy()
print('After handling missing values', df.shape)
x_train, x_test, y_train, y_test = pp.create_splits(df, 'collusion')

mask = pp.remove_outliers(x_train, 0.10)
x_train, y_train = x_train.iloc[mask, :], y_train.iloc[mask]
print('After handling outliers', x_train.shape)

(38567, 38)
(148720, 39)
(187287, 39)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:1565: RuntimeWarning: invalid value encountered in subtract
  X -= self.center_


Before handling missing values (27410, 23)
After handling missing values (23362, 23)
Shape of x and y before split (23362, 22) (23362,)
Shape of x_train and x_test after split (18689, 22) (4673, 22)
Train set contains 6.008882230188881 pct ratio of collusion labels
Test set contains 5.328482773378986 pct ratio of collusion labels



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


After handling outliers (16822, 22)


# Neural Network

In [ ]:
# credit: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

def recall_metric(y_true, y_pred):
  '''Takes predictions and original labels and outputs calculated recall score.'''
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
    
  return recall


def precision_metric(y_true, y_pred):
  '''Takes predictions and original labels and outputs calculated precision score.'''

  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
    
  return precision


def f1_metric(y_true, y_pred):
  '''Takes predictions and original labels and outputs calculated f-score, based 
  on harmonised mean of precision and recall.'''

  precision = precision_metric(y_true, y_pred)
  recall = recall_metric(y_true, y_pred)
    
  return 2*((precision*recall)/(precision+recall+K.epsilon()))


def balanced_acc(y_true, y_pred):

  accuracy = balacc = balanced_accuracy_score(y_true, y_pred)

  return accuracy

In [ ]:
loss, f1_score, recall_score, precision_score= model.evaluate(x_test, y_test)

In [ ]:
sns.lineplot(x=history.epoch, y=history.history['loss'])

In [ ]:
sns.lineplot(x=history.epoch, y=history.history['f1_metric'])

In [ ]:
sns.lineplot(x=history.epoch, y=history.history['recall_metric'])

In [ ]:
sns.lineplot(x=history.epoch, y=history.history['precision_metric'])

#Neural Network

In [ ]:
def build_network(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[22]): 
  '''Returns a multilayer network structure based on passed arguments for number 
  of hidden layers, neurons per hidden layer (same in all), learing rat and dimensions of input.'''
  
  model = keras.models.Sequential()
  options = {"input_shape": input_shape}
  
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
    options = {} 
  
  model.add(keras.layers.Dense(1, activation="sigmoid")) 
  optimizer = keras.optimizers.SGD(learning_rate) 

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision_metric, recall_metric, f1_metric])

  return model


param_distribs = {  
    "n_hidden": [1, 2, 3, 4],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

In [ ]:
nn_classifier = keras.wrappers.scikit_learn.KerasClassifier(build_network)
nn_random_search = RandomizedSearchCV(nn_classifier, param_distribs, n_iter=10, cv=3)
nn_random_search.fit(x_train, y_train, epochs=100, validation_split=0.3, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


246/246 [==============================] - 2s 5ms/step - loss: 0.2530 - precision_metric: 2.9036e-04 - recall_metric: 0.0041 - f1_metric: 5.4201e-04 - val_loss: 0.2238 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 2/100
246/246 [==============================] - 1s 6ms/step - loss: 0.2245 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2120 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 3/100
246/246 [==============================] - 2s 8ms/step - loss: 0.2117 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2025 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.1944 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


246/246 [==============================] - 2s 4ms/step - loss: 0.2327 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2223 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 2/100
246/246 [==============================] - 1s 3ms/step - loss: 0.2067 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2075 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 3/100
246/246 [==============================] - 1s 3ms/step - loss: 0.1914 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.1954 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.1803 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


246/246 [==============================] - 2s 5ms/step - loss: 0.2431 - precision_metric: 0.0014 - recall_metric: 0.0020 - f1_metric: 0.0016 - val_loss: 0.2470 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 2/100
246/246 [==============================] - 1s 3ms/step - loss: 0.2172 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2315 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 3/100
246/246 [==============================] - 1s 3ms/step - loss: 0.2042 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2137 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.1898 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.2241 - v

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-fin

368/368 [==============================] - 3s 4ms/step - loss: 0.2442 - precision_metric: 0.0011 - recall_metric: 0.0053 - f1_metric: 0.0018 - val_loss: 0.2092 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 2/100
368/368 [==============================] - 1s 4ms/step - loss: 0.2103 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.1904 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 3/100
368/368 [==============================] - 1s 4ms/step - loss: 0.1897 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.1842 - val_precision_metric: 0.0000e+00 - val_recall_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 4/100
368/368 [==============================] - 1s 4ms/step - loss: 0.1780 - precision_metric: 0.0000e+00 - recall_metric: 0.0000e+00 - f1_metric: 0.0000e+00 - val_loss: 0.1825 - v

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2c5de9e750>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2c6344b5d0>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [ ]:
print(nn_random_search.best_params_)
print(nn_random_search.best_score_)

model = nn_random_search.best_estimator_.model

loss = model.evaluate(x_test, y_test)

{'learning_rate': 0.0003438804075235824, 'n_hidden': 3, 'n_neurons': 69}
nan
147/147 [==============================] - 1s 2ms/step - loss: 0.1491 - precision_metric: 0.0136 - recall_metric: 0.0051 - f1_metric: 0.0073


#Weighted Neural Network

In [ ]:
weights = compute_class_weight(class_weight = "balanced", classes = np.unique(y_train), y = y_train)

weights = dict(zip(np.unique(y_train), weights))
print(weights)

balanced_classifier = keras.wrappers.scikit_learn.KerasClassifier(build_network)
nn_balanced = RandomizedSearchCV(balanced_classifier, param_distribs, n_iter=10, cv=3)
nn_balanced.fit(x_train, y_train, epochs=100, validation_split=0.3, class_weight=weights, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

{0.0: 0.5330544463459467, 1.0: 8.063279002876317}
Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


246/246 [==============================] - 2s 5ms/step - loss: 0.6866 - precision_metric: 0.0776 - recall_metric: 0.5129 - f1_metric: 0.1233 - val_loss: 0.4087 - val_precision_metric: 0.1236 - val_recall_metric: 0.1486 - val_f1_metric: 0.1191
Epoch 2/100
246/246 [==============================] - 1s 3ms/step - loss: 0.5823 - precision_metric: 0.1289 - recall_metric: 0.6974 - f1_metric: 0.2011 - val_loss: 0.4579 - val_precision_metric: 0.1489 - val_recall_metric: 0.7042 - val_f1_metric: 0.2366
Epoch 3/100
246/246 [==============================] - 1s 3ms/step - loss: 0.4843 - precision_metric: 0.1502 - recall_metric: 0.7866 - f1_metric: 0.2405 - val_loss: 0.4782 - val_precision_metric: 0.1460 - val_recall_metric: 0.7761 - val_f1_metric: 0.2382
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.4306 - precision_metric: 0.1695 - recall_metric: 0.8042 - f1_metric: 0.2691 - val_loss: 0.5579 - val_precision_metric: 0.1471 - val_recall_metric: 0.8354 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


246/246 [==============================] - 2s 5ms/step - loss: 0.6605 - precision_metric: 0.0535 - recall_metric: 0.3128 - f1_metric: 0.0822 - val_loss: 0.5318 - val_precision_metric: 0.0973 - val_recall_metric: 0.3835 - val_f1_metric: 0.1494
Epoch 2/100
246/246 [==============================] - 1s 4ms/step - loss: 0.5749 - precision_metric: 0.1227 - recall_metric: 0.6460 - f1_metric: 0.1830 - val_loss: 0.5535 - val_precision_metric: 0.1242 - val_recall_metric: 0.7906 - val_f1_metric: 0.2084
Epoch 3/100
246/246 [==============================] - 1s 5ms/step - loss: 0.4722 - precision_metric: 0.1403 - recall_metric: 0.7401 - f1_metric: 0.2239 - val_loss: 0.3735 - val_precision_metric: 0.1783 - val_recall_metric: 0.6987 - val_f1_metric: 0.2722
Epoch 4/100
246/246 [==============================] - 1s 4ms/step - loss: 0.4004 - precision_metric: 0.1679 - recall_metric: 0.7783 - f1_metric: 0.2650 - val_loss: 0.5134 - val_precision_metric: 0.1562 - val_recall_metric: 0.8632 - val_f1_metric:

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


246/246 [==============================] - 2s 4ms/step - loss: 0.6857 - precision_metric: 0.0753 - recall_metric: 0.3436 - f1_metric: 0.1172 - val_loss: 0.6667 - val_precision_metric: 0.0964 - val_recall_metric: 0.3347 - val_f1_metric: 0.1417
Epoch 2/100
246/246 [==============================] - 1s 3ms/step - loss: 0.6715 - precision_metric: 0.0889 - recall_metric: 0.3248 - f1_metric: 0.1313 - val_loss: 0.6669 - val_precision_metric: 0.1164 - val_recall_metric: 0.4772 - val_f1_metric: 0.1807
Epoch 3/100
246/246 [==============================] - 1s 3ms/step - loss: 0.6419 - precision_metric: 0.1118 - recall_metric: 0.4644 - f1_metric: 0.1672 - val_loss: 0.5798 - val_precision_metric: 0.1432 - val_recall_metric: 0.5382 - val_f1_metric: 0.2177
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.5615 - precision_metric: 0.1373 - recall_metric: 0.6471 - f1_metric: 0.2122 - val_loss: 0.4177 - val_precision_metric: 0.1806 - val_recall_metric: 0.5371 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-fin

368/368 [==============================] - 3s 5ms/step - loss: 0.6574 - precision_metric: 0.0902 - recall_metric: 0.5492 - f1_metric: 0.1438 - val_loss: 0.7243 - val_precision_metric: 0.0931 - val_recall_metric: 0.7741 - val_f1_metric: 0.1623
Epoch 2/100
368/368 [==============================] - 1s 4ms/step - loss: 0.4945 - precision_metric: 0.1463 - recall_metric: 0.7650 - f1_metric: 0.2342 - val_loss: 0.3484 - val_precision_metric: 0.1739 - val_recall_metric: 0.6698 - val_f1_metric: 0.2659
Epoch 3/100
368/368 [==============================] - 1s 4ms/step - loss: 0.4170 - precision_metric: 0.1730 - recall_metric: 0.8410 - f1_metric: 0.2743 - val_loss: 0.3729 - val_precision_metric: 0.1882 - val_recall_metric: 0.7585 - val_f1_metric: 0.2907
Epoch 4/100
368/368 [==============================] - 1s 4ms/step - loss: 0.3723 - precision_metric: 0.1881 - recall_metric: 0.8729 - f1_metric: 0.2983 - val_loss: 0.4283 - val_precision_metric: 0.1868 - val_recall_metric: 0.8544 - val_f1_metric:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2c5e5aaf50>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2c6344b5d0>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [ ]:
print(nn_balanced.best_params_)
print(nn_balanced.best_score_)

model_balanced = nn_balanced.best_estimator_.model

loss_balanced = model_balanced.evaluate(x_test, y_test)

{'learning_rate': 0.009567330683329616, 'n_hidden': 4, 'n_neurons': 80}
nan
147/147 [==============================] - 1s 2ms/step - loss: 0.3598 - precision_metric: 0.1753 - recall_metric: 0.8002 - f1_metric: 0.2791


#Weighted Neural Network (defined)

In [ ]:
weights2 = {0: 0.3,
            1: 80.}
balanced_classifier2 = keras.wrappers.scikit_learn.KerasClassifier(build_network)
nn_balanced2 = RandomizedSearchCV(balanced_classifier2, param_distribs, n_iter=10, cv=3)
nn_balanced2.fit(x_train, y_train, epochs=100, validation_split=0.3, class_weight=weights2, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


246/246 [==============================] - 2s 4ms/step - loss: 1.2833 - precision_metric: 0.0644 - recall_metric: 0.8943 - f1_metric: 0.1182 - val_loss: 2.5470 - val_precision_metric: 0.0607 - val_recall_metric: 0.8774 - val_f1_metric: 0.1116
Epoch 2/100
246/246 [==============================] - 1s 3ms/step - loss: 1.0172 - precision_metric: 0.0643 - recall_metric: 0.8862 - f1_metric: 0.1182 - val_loss: 2.5254 - val_precision_metric: 0.0609 - val_recall_metric: 0.8774 - val_f1_metric: 0.1119
Epoch 3/100
246/246 [==============================] - 1s 3ms/step - loss: 0.8723 - precision_metric: 0.0701 - recall_metric: 0.8780 - f1_metric: 0.1274 - val_loss: 1.9342 - val_precision_metric: 0.0725 - val_recall_metric: 0.8774 - val_f1_metric: 0.1314
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.7200 - precision_metric: 0.0875 - recall_metric: 0.8618 - f1_metric: 0.1551 - val_loss: 1.8235 - val_precision_metric: 0.0874 - val_recall_metric: 0.8774 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


246/246 [==============================] - 2s 4ms/step - loss: 1.2814 - precision_metric: 0.0581 - recall_metric: 0.8862 - f1_metric: 0.1072 - val_loss: 2.6062 - val_precision_metric: 0.0607 - val_recall_metric: 0.8774 - val_f1_metric: 0.1116
Epoch 2/100
246/246 [==============================] - 1s 4ms/step - loss: 1.0084 - precision_metric: 0.0581 - recall_metric: 0.8537 - f1_metric: 0.1068 - val_loss: 2.3409 - val_precision_metric: 0.0607 - val_recall_metric: 0.8774 - val_f1_metric: 0.1116
Epoch 3/100
246/246 [==============================] - 1s 4ms/step - loss: 0.9279 - precision_metric: 0.0585 - recall_metric: 0.8374 - f1_metric: 0.1073 - val_loss: 2.1287 - val_precision_metric: 0.0636 - val_recall_metric: 0.8774 - val_f1_metric: 0.1164
Epoch 4/100
246/246 [==============================] - 1s 3ms/step - loss: 0.8168 - precision_metric: 0.0658 - recall_metric: 0.8659 - f1_metric: 0.1202 - val_loss: 1.7616 - val_precision_metric: 0.0761 - val_recall_metric: 0.8774 - val_f1_metric:

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


246/246 [==============================] - 3s 5ms/step - loss: 1.3487 - precision_metric: 0.0614 - recall_metric: 0.8445 - f1_metric: 0.1122 - val_loss: 2.5422 - val_precision_metric: 0.0690 - val_recall_metric: 0.8774 - val_f1_metric: 0.1258
Epoch 2/100
246/246 [==============================] - 1s 4ms/step - loss: 1.0093 - precision_metric: 0.0608 - recall_metric: 0.8577 - f1_metric: 0.1116 - val_loss: 2.3601 - val_precision_metric: 0.0690 - val_recall_metric: 0.8774 - val_f1_metric: 0.1258
Epoch 3/100
246/246 [==============================] - 1s 4ms/step - loss: 0.8630 - precision_metric: 0.0673 - recall_metric: 0.8699 - f1_metric: 0.1226 - val_loss: 1.7853 - val_precision_metric: 0.0852 - val_recall_metric: 0.8774 - val_f1_metric: 0.1524
Epoch 4/100
246/246 [==============================] - 1s 4ms/step - loss: 0.6924 - precision_metric: 0.0871 - recall_metric: 0.8821 - f1_metric: 0.1551 - val_loss: 1.4495 - val_precision_metric: 0.1120 - val_recall_metric: 0.8774 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-fin

368/368 [==============================] - 2s 4ms/step - loss: 1.3208 - precision_metric: 0.0631 - recall_metric: 0.8537 - f1_metric: 0.1154 - val_loss: 2.6161 - val_precision_metric: 0.0591 - val_recall_metric: 0.8544 - val_f1_metric: 0.1089
Epoch 2/100
368/368 [==============================] - 1s 3ms/step - loss: 0.9559 - precision_metric: 0.0641 - recall_metric: 0.8505 - f1_metric: 0.1169 - val_loss: 1.7278 - val_precision_metric: 0.0678 - val_recall_metric: 0.8544 - val_f1_metric: 0.1235
Epoch 3/100
368/368 [==============================] - 1s 3ms/step - loss: 0.7579 - precision_metric: 0.0821 - recall_metric: 0.8940 - f1_metric: 0.1472 - val_loss: 1.6844 - val_precision_metric: 0.0885 - val_recall_metric: 0.8544 - val_f1_metric: 0.1569
Epoch 4/100
368/368 [==============================] - 1s 3ms/step - loss: 0.6003 - precision_metric: 0.1079 - recall_metric: 0.8641 - f1_metric: 0.1866 - val_loss: 1.3331 - val_precision_metric: 0.1137 - val_recall_metric: 0.8544 - val_f1_metric:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2c58a3a310>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2c6344b5d0>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [ ]:
print(nn_balanced2.best_params_)
print(nn_balanced2.best_score_)

model_balanced2 = nn_balanced2.best_estimator_.model

loss_balanced2 = model_balanced2.evaluate(x_test, y_test)

{'learning_rate': 0.0010633812553568715, 'n_hidden': 2, 'n_neurons': 58}
nan
147/147 [==============================] - 1s 2ms/step - loss: 0.9400 - precision_metric: 0.1732 - recall_metric: 0.8065 - f1_metric: 0.2769


#Neural Network with upsampling

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)
x_train_upsampled, y_train_upsampled = pp.oversample(x_train, y_train)

nn_classifier_upsampled = keras.wrappers.scikit_learn.KerasClassifier(build_network)
nn_upsampled = RandomizedSearchCV(nn_classifier_upsampled, param_distribs, n_iter=10, cv=3)
nn_upsampled.fit(x_train_upsampled, y_train_upsampled, epochs=100, validation_data=(x_val, y_val), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Counter({0.0: 6681, 1.0: 415})
Counter({0.0: 6681, 1.0: 6681})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  after removing the cwd from sys.path.


Epoch 1/100
279/279 [==============================] - 2s 4ms/step - loss: 0.5181 - precision_metric: 0.7646 - recall_metric: 0.9801 - f1_metric: 0.8547 - val_loss: 0.6694 - val_precision_metric: 0.1086 - val_recall_metric: 0.6651 - val_f1_metric: 0.1821
Epoch 2/100
279/279 [==============================] - 1s 3ms/step - loss: 0.3252 - precision_metric: 0.8734 - recall_metric: 0.9731 - f1_metric: 0.9185 - val_loss: 0.8567 - val_precision_metric: 0.1143 - val_recall_metric: 0.8333 - val_f1_metric: 0.1973
Epoch 3/100
279/279 [==============================] - 1s 3ms/step - loss: 0.2557 - precision_metric: 0.9046 - recall_metric: 0.9881 - f1_metric: 0.9432 - val_loss: 0.7204 - val_precision_metric: 0.1566 - val_recall_metric: 0.8018 - val_f1_metric: 0.2545
Epoch 4/100
279/279 [==============================] - 1s 3ms/step - loss: 0.2368 - precision_metric: 0.9130 - recall_metric: 0.9927 - f1_metric: 0.9503 - val_loss: 0.5596 - val_precision_metric: 0.1730 - val_recall_metric: 0.7180 - va

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


279/279 [==============================] - 2s 4ms/step - loss: 0.6140 - precision_metric: 0.6520 - recall_metric: 0.7524 - f1_metric: 0.6831 - val_loss: 0.6503 - val_precision_metric: 0.1081 - val_recall_metric: 0.7101 - val_f1_metric: 0.1842
Epoch 2/100
279/279 [==============================] - 1s 3ms/step - loss: 0.4067 - precision_metric: 0.7827 - recall_metric: 0.9105 - f1_metric: 0.8339 - val_loss: 0.4391 - val_precision_metric: 0.1656 - val_recall_metric: 0.6984 - val_f1_metric: 0.2576
Epoch 3/100
279/279 [==============================] - 1s 3ms/step - loss: 0.3274 - precision_metric: 0.8227 - recall_metric: 0.9524 - f1_metric: 0.8791 - val_loss: 0.5996 - val_precision_metric: 0.1652 - val_recall_metric: 0.7833 - val_f1_metric: 0.2643
Epoch 4/100
279/279 [==============================] - 1s 3ms/step - loss: 0.3107 - precision_metric: 0.8278 - recall_metric: 0.9695 - f1_metric: 0.8890 - val_loss: 0.5070 - val_precision_metric: 0.1648 - val_recall_metric: 0.7800 - val_f1_metric:

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


279/279 [==============================] - 2s 4ms/step - loss: 0.5006 - precision_metric: 0.1997 - recall_metric: 0.0852 - f1_metric: 0.1042 - val_loss: 0.2964 - val_precision_metric: 0.1936 - val_recall_metric: 0.3437 - val_f1_metric: 0.2360
Epoch 2/100
279/279 [==============================] - 1s 3ms/step - loss: 0.3620 - precision_metric: 0.6224 - recall_metric: 0.5859 - f1_metric: 0.5696 - val_loss: 0.3328 - val_precision_metric: 0.1735 - val_recall_metric: 0.6342 - val_f1_metric: 0.2593
Epoch 3/100
279/279 [==============================] - 1s 3ms/step - loss: 0.3245 - precision_metric: 0.6330 - recall_metric: 0.7159 - f1_metric: 0.6377 - val_loss: 0.2225 - val_precision_metric: 0.1854 - val_recall_metric: 0.3437 - val_f1_metric: 0.2177
Epoch 4/100
279/279 [==============================] - 1s 3ms/step - loss: 0.2930 - precision_metric: 0.6341 - recall_metric: 0.8114 - f1_metric: 0.6952 - val_loss: 0.2331 - val_precision_metric: 0.1914 - val_recall_metric: 0.3730 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-fin

418/418 [==============================] - 2s 4ms/step - loss: 0.5253 - precision_metric: 0.6993 - recall_metric: 0.7777 - f1_metric: 0.7180 - val_loss: 0.6282 - val_precision_metric: 0.1215 - val_recall_metric: 0.7901 - val_f1_metric: 0.2061
Epoch 2/100
418/418 [==============================] - 1s 3ms/step - loss: 0.3381 - precision_metric: 0.8027 - recall_metric: 0.9481 - f1_metric: 0.8642 - val_loss: 0.4509 - val_precision_metric: 0.1712 - val_recall_metric: 0.7800 - val_f1_metric: 0.2718
Epoch 3/100
418/418 [==============================] - 1s 3ms/step - loss: 0.3075 - precision_metric: 0.8115 - recall_metric: 0.9719 - f1_metric: 0.8809 - val_loss: 0.3820 - val_precision_metric: 0.1716 - val_recall_metric: 0.6200 - val_f1_metric: 0.2596
Epoch 4/100
418/418 [==============================] - 1s 3ms/step - loss: 0.2919 - precision_metric: 0.8193 - recall_metric: 0.9789 - f1_metric: 0.8887 - val_loss: 0.4942 - val_precision_metric: 0.1707 - val_recall_metric: 0.7203 - val_f1_metric:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0ebc232790>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0eaf8172d0>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [ ]:
print(nn_upsampled.best_params_)
print(nn_upsampled.best_score_)

model_upsampled = nn_upsampled.best_estimator_.model

loss = model_upsampled.evaluate(x_test, y_test)

{'learning_rate': 0.00030280976129781783, 'n_hidden': 4, 'n_neurons': 87}
nan
147/147 [==============================] - 0s 2ms/step - loss: 0.3900 - precision_metric: 0.1646 - recall_metric: 0.5949 - f1_metric: 0.2459


# Neural Network with upsampling and weighting

In [ ]:
balanced_classifier3 = keras.wrappers.scikit_learn.KerasClassifier(build_network)
nn_balanced3 = RandomizedSearchCV(balanced_classifier3, param_distribs, n_iter=10, cv=3)
nn_balanced3.fit(x_train_upsampled, y_train_upsampled, epochs=100, validation_split=0.3, class_weight=weights, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


461/461 [==============================] - 3s 4ms/step - loss: 0.9035 - precision_metric: 0.5964 - recall_metric: 1.0000 - f1_metric: 0.7435 - val_loss: 0.0322 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric: 1.0000
Epoch 2/100
461/461 [==============================] - 2s 3ms/step - loss: 0.6067 - precision_metric: 0.6605 - recall_metric: 1.0000 - f1_metric: 0.7913 - val_loss: 0.0259 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric: 1.0000
Epoch 3/100
461/461 [==============================] - 2s 3ms/step - loss: 0.4011 - precision_metric: 0.7707 - recall_metric: 0.9999 - f1_metric: 0.8676 - val_loss: 0.0129 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric: 1.0000
Epoch 4/100
461/461 [==============================] - 2s 4ms/step - loss: 0.3371 - precision_metric: 0.8296 - recall_metric: 1.0000 - f1_metric: 0.9049 - val_loss: 0.0089 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


461/461 [==============================] - 3s 4ms/step - loss: 1.1118 - precision_metric: 0.3314 - recall_metric: 0.9929 - f1_metric: 0.4903 - val_loss: 0.1505 - val_precision_metric: 1.0000 - val_recall_metric: 0.9928 - val_f1_metric: 0.9963
Epoch 2/100
461/461 [==============================] - 1s 3ms/step - loss: 0.7803 - precision_metric: 0.4186 - recall_metric: 0.9960 - f1_metric: 0.5809 - val_loss: 0.0771 - val_precision_metric: 1.0000 - val_recall_metric: 0.9986 - val_f1_metric: 0.9993
Epoch 3/100
461/461 [==============================] - 1s 3ms/step - loss: 0.5324 - precision_metric: 0.5456 - recall_metric: 1.0000 - f1_metric: 0.6975 - val_loss: 0.0492 - val_precision_metric: 1.0000 - val_recall_metric: 0.9998 - val_f1_metric: 0.9999
Epoch 4/100
461/461 [==============================] - 1s 3ms/step - loss: 0.4500 - precision_metric: 0.6299 - recall_metric: 0.9997 - f1_metric: 0.7648 - val_loss: 0.0307 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric:

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,


461/461 [==============================] - 3s 4ms/step - loss: 0.6669 - precision_metric: 0.0947 - recall_metric: 0.4609 - f1_metric: 0.1473 - val_loss: 0.6299 - val_precision_metric: 0.7007 - val_recall_metric: 0.5464 - val_f1_metric: 0.5318
Epoch 2/100
461/461 [==============================] - 1s 3ms/step - loss: 0.6003 - precision_metric: 0.1279 - recall_metric: 0.6451 - f1_metric: 0.2019 - val_loss: 0.6018 - val_precision_metric: 0.7198 - val_recall_metric: 0.5543 - val_f1_metric: 0.5707
Epoch 3/100
461/461 [==============================] - 1s 3ms/step - loss: 0.5361 - precision_metric: 0.1430 - recall_metric: 0.7462 - f1_metric: 0.2308 - val_loss: 0.4717 - val_precision_metric: 0.7118 - val_recall_metric: 0.9093 - val_f1_metric: 0.7250
Epoch 4/100
461/461 [==============================] - 1s 3ms/step - loss: 0.4850 - precision_metric: 0.1569 - recall_metric: 0.7853 - f1_metric: 0.2517 - val_loss: 0.4461 - val_precision_metric: 0.7234 - val_recall_metric: 0.8919 - val_f1_metric:

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 323, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-fin

691/691 [==============================] - 4s 4ms/step - loss: 1.0227 - precision_metric: 0.2991 - recall_metric: 0.9970 - f1_metric: 0.4533 - val_loss: 0.1148 - val_precision_metric: 1.0000 - val_recall_metric: 0.9995 - val_f1_metric: 0.9997
Epoch 2/100
691/691 [==============================] - 3s 4ms/step - loss: 0.5862 - precision_metric: 0.4699 - recall_metric: 0.9985 - f1_metric: 0.6287 - val_loss: 0.0526 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric: 1.0000
Epoch 3/100
691/691 [==============================] - 3s 4ms/step - loss: 0.4590 - precision_metric: 0.5796 - recall_metric: 0.9998 - f1_metric: 0.7250 - val_loss: 0.0434 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric: 1.0000
Epoch 4/100
691/691 [==============================] - 3s 4ms/step - loss: 0.4326 - precision_metric: 0.5947 - recall_metric: 0.9995 - f1_metric: 0.7374 - val_loss: 0.0424 - val_precision_metric: 1.0000 - val_recall_metric: 1.0000 - val_f1_metric:

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2c5ba7de10>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2c6344b5d0>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [ ]:
print(nn_balanced3.best_params_)
print(nn_balanced3.best_score_)

model_balanced3 = nn_balanced3.best_estimator_.model

loss = model_balanced3.evaluate(x_test, y_test)

{'learning_rate': 0.024085700759503826, 'n_hidden': 3, 'n_neurons': 27}
nan
147/147 [==============================] - 1s 3ms/step - loss: 0.7731 - precision_metric: 0.1754 - recall_metric: 0.7988 - f1_metric: 0.2791


In [ ]:
print(loss)

[0.7730925679206848, 0.17541424930095673, 0.7987528443336487, 0.2790590822696686]
